# 파이토치 제공 데이터 사용

In [1]:
import torch
import torchvision
import torchvision.transforms as tr # 데이터 불러오면서 바로 전처리
from torch.utils.data import DataLoader, Dataset
import numpy as np

In [4]:
transf = tr.Compose([tr.Resize(8),tr.ToTensor()]) # (8,8)로 resize, tensor데이터로 변환

In [6]:
trainset = torchvision.datasets.CIFAR10(root='./data',train=True, download=True, transform=transf) #transform=전처리
testset = torchvision.datasets.CIFAR10(root='./data',train=False, download=True,transform=transf)

 99%|█████████▉| 169336832/170498071 [00:53<00:00, 1584889.64it/s]Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [7]:
trainset[0][0].size() # 이미지: 레이블(튜플형태로 들어가 있음)
# (3개 채널(R, G, B), 사이즈)

torch.Size([3, 8, 8])

# DataLoader이용해서 배치 사이즈로 변경

In [8]:
trainloader = DataLoader(trainset, batch_size=50, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=50, shuffle=True, num_workers=2)

In [9]:
len(trainloader) #CIFAR10 train data개수가 50,000개를 batch_size=50으로 나눈 개수

1000

# dataiter, next ===> data하나씩 불러올 때

In [11]:
# data하나씩 불러올때
dataiter = iter(trainloader)
images, labels = dataiter.next()

In [12]:
images.size() # 배치사이즈, 채널수, 이미지 너비, 이미지 높이

torch.Size([50, 3, 8, 8])

# 폴더 내 데이터 불러올 때

In [15]:
# 같은 클래스별 폴더 이미지 데이터 이용
#./class/tiger   ./class/lion
transf = tr.Compose([tr.Resize(8), tr.ToTensor()])
trainset = torchvision.datasets.ImageFolder(root='./class',transform=transf) 
# torchvision.datasets.ImageFolder이용하면 클래스 안에 있는 image들을 알아서 search해줌. 각각의 다른 폴더에 대해서 레이블을 다르게 자동으로 매겨줌. 바로 전처리도 이용 가능(transform이용해서)
trainloader = DataLoader(trainset, batch_size=1, shuffle=False, num_workers=2)
print(len(trainloader))

FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: './class'

# 직접 데이터 불러오기

In [16]:
train_images = np.random.randint(256,size=(20,32,32,3))
train_labels = np.random.randint(2,size=(20,1))

print(train_images.shape, train_labels.shape)

(20, 32, 32, 3) (20, 1)


# 이 양식은 꼭 기억해놓을 것

In [18]:
class TensorData(Dataset): # 상속받을 클래스
    def __init__(self, x_data,y_data): # 외부에서 들어올 데이터
        self.x_data = torch.FloatTensor(x_data) # tensor로 변환   # 이미지 개수, 채널 수, 이미지 너비, 높이(원래는 이미지 개수, 이미지 너비, 높이, 채널 수)
        self.x_data  = self.x_data.permute(0,3,1,2)
        self.y_data = torch.LongTensor(y_data) # Float, Long이냐 정할 수 있다.
        self.len = self.y_data.shape[0] # 데이터 개수

    def __getitem__(self, index):
        return self.x_data[index],self.y_data[index] # x,y를 튜플 형태로 밖으로 내보냄

    def __len__(self):
        return self.len

In [19]:
train_data = TensorData(train_images,train_labels)
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)

In [22]:
train_data[0][0].size() # 튜플 형태이기 때문에

torch.Size([3, 32, 32])

In [24]:
dataiter = iter(train_loader)
images, labels = dataiter.next()

In [25]:
images.size()

torch.Size([10, 3, 32, 32])

# 파이토치로 전처리

In [27]:
class MyDataset(Dataset):
    def __init__(self,x_data, y_data, transform=None): 
        self.x_data = x_data  # 텐서로 변환하지 않음
        self.y_data = y_data
        self.transform = transform
        self.len = len(y_data)
    def __getitem__(self,index):
        sample = self.x_data[index], self.y_data[index]

        if self.transform:                 # 튜플 형태로 내보내기 전에 전처리
            sample = self.transform(sample)

        return sample
    def __len__(self):
        return self.len

class ToTensor:  # 텐서변환
    def __call__(self,sample):
        inputs, labels = sample
        inputs = torch.FloatTensor(inputs)
        inputs = inputs.permute(2,0,1)
        return inputs, torch.LongTensor(labels)

class LinearTensor: # linear형태로 변환
    def __init__(self, slope=1, bias=0):
        self.slope = slope
        self.bias = bias
    
    def __call__(self,sample):
        inputs, labels = sample
        inputs = self.slope*inputs + self.bias
        return inputs, labels


In [28]:
trans = tr.Compose([ToTensor(), LinearTensor(2,5)]) # TonTensor왜 만드냐...PIL images여만 사용가능하기 때문데(tr.ToTensor()이용하면)
ds1 = MyDataset(train_images, train_labels, transform=trans) # 튜플형태
train_loader1 = DataLoader(ds1, batch_size=10, shuffle=True)

In [30]:
first_data = ds1[0]
features, labels = first_data
print(type(features),type(labels))

<class 'torch.Tensor'> <class 'torch.Tensor'>


In [32]:
dataiter1 = iter(train_loader1)
images1, labels1 = dataiter1.next()

In [33]:
images1

tensor([[[[155.,  23., 333.,  ..., 247., 493.,  59.],
          [169., 211.,  49.,  ..., 437., 437.,  91.],
          [345.,  79., 441.,  ...,  11., 289.,  11.],
          ...,
          [445., 487., 427.,  ..., 505., 465., 383.],
          [ 15., 397., 167.,  ..., 237., 275., 397.],
          [449., 201., 277.,  ..., 437., 477., 133.]],

         [[ 11., 429., 291.,  ...,  41., 263., 195.],
          [445., 199., 219.,  ..., 165., 235., 117.],
          [ 19., 167., 323.,  ..., 131.,  61., 329.],
          ...,
          [117., 277., 417.,  ..., 371., 411., 127.],
          [385.,  83., 351.,  ...,  49., 385., 251.],
          [ 31., 513.,  89.,  ..., 305., 145.,  37.]],

         [[ 13., 315.,  79.,  ..., 307., 239., 233.],
          [427., 477.,   5.,  ..., 107., 417., 155.],
          [395., 351.,  21.,  ...,  45., 375.,  55.],
          ...,
          [353., 177., 277.,  ..., 203., 141., 411.],
          [485., 241., 387.,  ...,  73.,  43., 485.],
          [235., 159., 433.,  ...

# tensor---->PIL image먼저 변환 후 tr.ToTensor()사용가능

In [46]:
class MyDataset(Dataset):
    def __init__(self,x_data, y_data, transform=None): 
        self.x_data = x_data  # 텐서로 변환하지 않음
        self.y_data = y_data
        self.transform = transform
        self.len = len(y_data)
    def __getitem__(self,index):
        sample = self.x_data[index], self.y_data[index]

        if self.transform:                 # 튜플 형태로 내보내기 전에 전처리
            sample = self.transform(sample)

        return sample
    def __len__(self):
        return self.len

class MyTransform:
    def __call__(self, sample):
        inputs, labels = sample
        inputs = torch.FloatTensor(inputs)
        inputs = inputs.permute(2,0,1)
        labels = torch.FloatTensor(labels)

        transf = tr.Compose([tr.ToPILImage(),tr.Resize(128),tr.ToTensor(), tr.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
        final_output = transf(inputs)

        return final_output, labels

In [47]:
ds2 = MyDataset(train_images, train_labels, transform = MyTransform())
train_loader2 = DataLoader(ds2, batch_size=10, shuffle=True) # batch형태 쓰는 이유는 전체 데이터 쓰면 너무 느리기 때문에 일부만 넣어서 학습햐려고

In [48]:
first_data=ds2[0]
features, labels = first_data

In [49]:
dataiter2 = iter(train_loader2)
images2, labels2 = dataiter2.next()

In [53]:
images2.shape

torch.Size([10, 3, 128, 128])

In [54]:
labels2.shape

torch.Size([10, 1])